In [ ]:
import os
import re
import sys
import glob
import pickle
import shelve
from collections import OrderedDict

from comet_ml.api import API, APIExperiment

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_slide
from dlml.nn import predict

In [ ]:
momentum = lambda H, S, fn=60.: 2 * H@S / fn * 1e-3

inertia = lambda H, S: H@S / S.sum()


def make_filename(H_values, data_dir, data_file_suffix='ieee39_PF_stoch_loads_compensators'):
    data_file = data_file_suffix
    variable_generators = []
    for gen in H_values[0].keys():
        h_val = np.array([v[gen] for v in H_values])
        if np.all(h_val == h_val[0]):
            data_file += f'_{h_val[0]:.3f}'
        else:
            variable_generators.append(gen)
            data_file += '_' + '-'.join(list(map(lambda s: f'{s:.3f}', h_val)))
    data_file += '.h5'
    return os.path.join(data_dir, data_file), variable_generators


def analyze_single_file_experiment(H_values, data_dir, var_names, data_mean, data_std,
                                   window_dur, window_step, area_id,
                                   data_file_suffix='ieee39_PF_stoch_loads_compensators'):
    data_file,var_gens = make_filename(H_values, data_dir, data_file_suffix)
    if not os.path.isfile(data_file):
        raise Exception(data_file + ': no such file')
    data_time, data, data_normalized, data_sliding, _ = load_data_slide([data_file],
                                                                        var_names,
                                                                        data_mean,
                                                                        data_std,
                                                                        window_dur,
                                                                        window_step,
                                                                        add_omega_ref = False,
                                                                        verbose = True)
    prediction_time, prediction, _ = predict(model, data_sliding, window_step)
        
    sim_dur = np.ceil(data_time[-1])
    N_H = len(H_values)
    block_dur = sim_dur / N_H
    S = np.array([P_nom_with_comp[gen]*1e-6 for gen in generators_areas_map_with_comp['default'][area_id-1]])
    area_inertia = np.zeros(N_H)
    exact = np.zeros(N_H)
    mean_prediction = np.zeros(N_H)
    for i in range(N_H):
        try:
            H = np.array([H_values[i]['G02'],
                          H_values[i]['G03'],
                          H_values[i]['Comp11']])
        except:
            H = np.array([H_values[i]['G02'],
                          H_values[i]['VSG03'],
                          H_values[i]['Comp11']])
        area_inertia[i] = inertia(H, S)
        exact[i] = momentum(H, S)
        idx = (prediction_time >= i*block_dur) & (prediction_time < (i+1)*block_dur)
        mean_prediction[i] = np.nanmean(prediction[idx])
        print('H = {} -> H = {:.4f} s, M = {:.4f}, M_pred = {:.4f} GW.s2'.format(
            H, area_inertia[i], exact[i], mean_prediction[i]))

    return data_time, data, data_normalized, prediction_time, prediction, area_inertia, exact, mean_prediction


def plot_single_file_experiment(time, prediction, exact, mean_prediction, N_blocks,
                                block_dur, area_measure, measure_units):
    fig,ax = plt.subplots(1, 1, figsize=(6,3))
    for i in range(N_blocks):
        t0,t1 = block_dur*i/60, block_dur*(i+1)/60
        ax.plot([t0, t1], mean_prediction[i] + np.zeros(2), 'k--', lw=1)
        ax.plot([t0, t1], exact[i] + np.zeros(2), 'r--', lw=1)
    ax.plot(time/60, prediction, 'k', lw=2)
    for side in 'right','top':
        ax.spines[side].set_visible(False)
    ax.set_xlabel('Time [min]')
    ax.set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
    ax.grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
    fig.tight_layout()
    

make_experiment_dict = lambda H_values, data_time, data, data_normalized, \
                            prediction_time, prediction, area_inertia, exact, \
                            mean_prediction, description='': \
    {
        'H_values': H_values,
        'data_time': data_time, 'data': data, 'data_normalized': data_normalized,
        'prediction_time': prediction_time, 'prediction': prediction,
        'area_inertia': area_inertia,
        'exact': exact,
        'mean_prediction': mean_prediction,
        'description': description
    }

#### Find the best experiment given a set of tags

In [ ]:
# full grid without varying compensator's inertia
# experiment_ID = '7d55f784f6b64f2caeb866804bda1a8b' # WRONG
experiment_ID = 'f64bde90cab54d1ea770bb21f33c3ed1'
# full grid with variable compensator's inertia
# experiment_ID = '57dfd307a5a945d8b28e5cf501b41f13' # WRONG
# experiment_ID = '13ea047db0cb40efa3e2743de9b9726e'
# experiment_ID = '98d1862b1cc1465884cf9e5ab59fc4da'
experiment_ID = 'a40658acee3c4e419c0ee34d0c59f4df'

workspace = 'danielelinaro'
project_name = 'inertia'

api = API(api_key = os.environ['COMET_API_KEY'])
experiment = api.get_experiment(workspace, project_name, experiment_ID)
sys.stdout.write(f'Getting metrics and tags for experiment {experiment_ID[:6]}... ')
sys.stdout.flush()
metrics = experiment.get_metrics()
tags = experiment.get_tags()
sys.stdout.write('done.\n')

print(f'Experiment {experiment_ID[:6]} has the following tags:')
nchar = 0
for tag in tags:
    if 'buses_' in tag:
        rec_bus_IDs = sorted(list(map(int, re.findall('\d+', tag))))
    elif 'area_measure_' in tag:
        area_measure = tag.split('_')[-1]
    elif 'area' in tag:
        area_ID = int(tag[4:])
    elif tag == 'all_stoch_loads':
        stoch_load_bus_IDs = []
    sys.stdout.write('"' + tag + '"   ')
    nchar += len(tag) + 5
    if nchar >= 70:
        nchar = 0
        sys.stdout.write('\n')

In [ ]:
blob = {}
metric_names = 'loss', 'val_loss', 'lr', 'epoch_duration'
for m in metrics:
    name = m['metricName']
    if name in metric_names:
        step = int(m['step'])
        if step not in blob:
            blob[step] = {}
        blob[step][name] = float(m['metricValue'])
        blob[step]['epoch'] = int(m['epoch'])
    elif name == 'mape_prediction':
        MAPE = float(m['metricValue'])
epochs = np.array([v['epoch'] for v in blob.values()]) + 1
data = {
    'step': np.array(list(blob.keys()))
}
for k in metric_names:
    data[k] = np.array([v[k] for v in blob.values()])
df = pd.DataFrame(data, epochs)
df.index.rename('epoch', inplace=True)
columns = df.columns.to_list()
columns[columns.index('lr')] = 'learning_rate'
df.columns = columns

In [ ]:
print('Experiment {} has minimum validation loss = {:.4f} and MAPE = {:.2f}%.'.format(
    experiment_ID[:6], df['val_loss'].min(), MAPE))

In [ ]:
Pfrac = 0.1
VSG_pars = {'KAD': 2.0, 'KW': 10.0}
shelf_name = experiment_ID[:6] + f'_Pfrac={Pfrac:.1f}'
for k,v in VSG_pars.items():
    shelf_name += f'_{k}={v:.1f}'
print(f'Saving data to {experiment_ID}/{shelf_name}.out')
db = shelve.open(os.path.join('..','experiments','neural_network', experiment_ID, shelf_name + '.out'),
                 flag='n')
db['metrics'] = df
db['MAPE'] = MAPE

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,3))

ax.plot(df.index, df['loss'], 'k', lw=1, label='Batch')
ax.plot(df.index, df['val_loss'], 'r', lw=1, label='Validation')
ax.grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.legend(loc='upper right', frameon=False)

twin = ax.twinx()
twin.plot(df.index, df['learning_rate'], color=[0,.2,.8], lw=1)
twin.set_ylabel('Learning rate')

fig.tight_layout()

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
if len(checkpoint_files) == 1:
    best_checkpoint = checkpoint_files[0]
else:
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=False)
model.compile()
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
data_dirs = [os.path.join('..', d.format(a)) if '{}' in d else os.path.join('..', d) \
             for d,a in zip(network_parameters['data_dirs'], network_parameters['area_IDs'])]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names'] if 'bus' in var_name]
var_names_fmt = OrderedDict({k + '{}': [] for k in tmp})
tmp = [re.findall('.*_line', var_name)[0] for var_name in network_parameters['var_names'] if 'line' in var_name]
for k in tmp:
    var_names_fmt[k + '_{}_{}'] = []
var_names_fmt = list(var_names_fmt.keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
var_names = [fmt.format(bus) for fmt in var_names_fmt for bus in rec_bus_IDs]
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')
db['x_train_mean'] = x_train_mean
db['x_train_std'] = x_train_std
db['data_dir'] = data_dir
db['weights_file'] = best_checkpoint
db['var_names_fmt'] = var_names_fmt

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

In [ ]:
default_H = OrderedDict([
    ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
    ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20)
])
default_H_with_comp = OrderedDict([
    ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
    ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
    ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
])
default_H_with_comp_and_vsg = OrderedDict([
    ('G01', 5.00), ('G02', 4.33), ('G04', 3.57), ('G05', 4.33),
    ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
    ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1), ('VSG03', 4.47)
])


generators_areas_map = {
    'default': [
        ['G02', 'G03'],
        ['G04', 'G05', 'G06', 'G07'],
        ['G08', 'G09', 'G10'],
        ['G01']
    ]
}
generators_areas_map_with_comp = {
    'default': [
        ['G02', 'G03', 'Comp11'],
        ['G04', 'G05', 'G06', 'G07', 'Comp21'],
        ['G08', 'G09', 'G10', 'Comp31'],
        ['G01']
    ]
}
generators_areas_map_with_comp_and_vsg = {
    'default': [
        ['G02', 'VSG03', 'Comp11'],
        ['G04', 'G05', 'G06', 'G07', 'Comp21'],
        ['G08', 'G09', 'G10', 'Comp31'],
        ['G01']
    ]
}

P_nom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04':  800e6, 'G05':  300e6,
         'G06':   800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6}
P_nom_with_comp = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04':  800e6, 'G05':  300e6,
                   'G06':   800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
                   'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}
P_nom_with_comp_and_vsg = {'G01': 10000e6, 'G02': 700e6, 'G04':  800e6, 'G05':  300e6,
                           'G06':   800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
                           'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6, 'VSG03': 800e6}


window_dur = 60
window_step = 1
var_names = network_parameters['var_names']
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

if area_measure == 'inertia':
    measure_units = 's'
elif area_measure == 'energy':
    measure_units = r'GW$\cdot$s'
elif area_measure == 'momentum':
    measure_units = r'GW$\cdot$s$^2$'
    
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))

abbrv = {'inertia': 'H', 'energy': 'E', 'momentum': 'M'}

EXPERIMENTS = []

db['window_dur'] = window_dur
db['window_step'] = window_step
db['var_names'] = var_names
db['data_mean'] = data_mean
db['data_std'] = data_std

### Steps of momentum varying only the inertia of the generators
`G03` has `type = 6`

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G03', 3.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
]

data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, data_mean, data_std,
                                   window_dur, window_step, area_ID)

N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                                       'momentum steps varying only generators inertia, G3 type=6'))

### Steps of momentum varying only the inertia of the generators

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G03', 3.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
]

data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, data_mean, data_std,
                                   window_dur, window_step, area_ID,
                                 data_file_suffix='ieee39_PF_stoch_loads_compensators_G3_type=2')

N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                                       'momentum steps varying only generators inertia, G3 type=2'))

### Different combinations of inertia corresponding to the same area momentum

In [ ]:
S = np.array([700, 800, 100])
H = np.array([4.33, 4.47, 0.1])
print('M =', momentum(H,S))
H = np.array([3.83, 4.9075, 0.1])
print('M =', momentum(H,S))
H = np.array([4.90142857, 3.97, 0.1])
print('M =', momentum(H,S))

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G03', 4.907), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.901), ('G03', 3.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
#     OrderedDict([
#         ('G01', 5.00), ('G02', 3.53), ('G03', 5.17), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
#         ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
#     ]),
#     OrderedDict([
#         ('G01', 5.00), ('G02', 5.244), ('G03', 3.67), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
#         ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
#     ])
#     OrderedDict([
#         ('G01', 5.00), ('G02', 3.93), ('G03', 4.87), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
#         ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
#     ]),
#     OrderedDict([
#         ('G01', 5.00), ('G02', 4.73), ('G03', 4.07), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
#         ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
#     ])
]

data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, data_mean, data_std,
                                   window_dur, window_step, area_ID)
N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                                       'fixed area momentum while varying generators inertia'))

### Same steps of area momentum obtained in two different ways:
Either by changing the inertia of the generators...

In [ ]:
S = np.array([700, 800, 100])
H = np.array([4.33, 4.47, 0.1])
print('M =', momentum(H,S))
H = np.array([4.24150442, 4.84743363, 0.1])
print('M =', momentum(H,S))
H = np.array([4.39637168, 5.02442478, 0.1])
print('M =', momentum(H,S))

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 4.242), ('G03', 4.847), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.396), ('G03', 5.024), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
#     OrderedDict([
#         ('G01', 5.00), ('G02', 4.58), ('G03', 4.72), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
#         ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
#     ]),
#     OrderedDict([
#         ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
#         ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
#     ])
]

data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, data_mean, data_std,
                                   window_dur, window_step, area_ID)

N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                                       'increasing momentum varying generators inertia'))

... or by changing the inertia of the compensator

In [ ]:
S = np.array([700, 800, 100])
H = np.array([4.33, 4.47, 0.1])
print('M =', momentum(H,S))
H = np.array([4.33, 4.47, 2.5])
print('M =', momentum(H,S))
H = np.array([4.33, 4.47, 5.0])
print('M =', momentum(H,S))

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
        ('Comp11', 2.5), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 5.0), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
#     OrderedDict([
#         ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
#         ('Comp11', 4.0), ('Comp21', 0.1), ('Comp31', 0.1)
#     ]),
#     OrderedDict([
#         ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
#         ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
#         ('Comp11', 8.0), ('Comp21', 0.1), ('Comp31', 0.1)
#     ])
]

data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, data_mean, data_std,
                                   window_dur, window_step, area_ID)

N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                                       'increasing momentum varying compensator inertia'))

### Steps of momentum varying only the inertia of the generators
The generator `G03` is split in 2, the first one (the original one) having `1-Pfrac` of the power and inertia, and the second one having `Pfrac` power and inertia.

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G03', 3.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
]

for gen_type in 2, 6:
    data_time, data, data_normalized, prediction_time, prediction, \
        area_inertia, exact, mean_prediction = \
        analyze_single_file_experiment(H_values, data_dir, var_names, None, data_std,
                                       window_dur, window_step, area_ID,
                                       data_file_suffix='ieee39_PF_stoch_loads_compensators_' + \
                                           f'split_G3_type={gen_type}_Pfrac={Pfrac:.1f}')

    N_blocks = len(H_values)
    block_dur = np.ceil(data_time[-1]) / N_blocks
    plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                                N_blocks, block_dur, area_measure, measure_units)

    EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                            prediction_time, prediction, area_inertia, exact, mean_prediction,
                                f'momentum steps varying only generators inertia, split G3 type={gen_type}'))

### Steps of momentum varying the inertia of one generator and one virtual synchronous generator
The generator `G03` is replaced by a VSG.

In [ ]:
H_values = [
    default_H_with_comp_and_vsg,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1), ('VSG03', 3.97)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.83), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1), ('VSG03', 4.97)
    ])
]

suffix = 'ieee39_PF_stoch_loads_compensators_vsg'
for k,v in VSG_pars.items():
    suffix += f'_{k}={v:.1f}'
print(suffix)
data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, None, data_std,
                                   window_dur, window_step, area_ID, data_file_suffix=suffix)

N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                            'momentum steps varying inertia of G2 and VSG3, a VSG that replaces G3'))

### Steps of momentum varying the inertia of two generators and one virtual synchronous generator
The VSG accounts for `Pfrac` of the power and inertia of the generator `G03`.

In [ ]:
def print_momentum(H,S):
    sys.stdout.write('H = [')
    for h in H:
        sys.stdout.write(f'{h:.2f} ')
    sys.stdout.write(f'] -> M = {momentum(H,S):.4f} GW.s2\n')

S = np.array([700, 800, 800, 100])
H = np.array([4.33, 4.47*(1-Pfrac), 4.47*Pfrac, 0.1])
print_momentum(H,S)
H = np.array([3.83, 3.97*(1-Pfrac), 3.97*Pfrac, 0.1])
print_momentum(H,S)
H = np.array([4.83, 4.97*(1-Pfrac), 4.97*Pfrac, 0.1])
print_momentum(H,S)

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G03', 3.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
]

data_time, data, data_normalized, prediction_time, prediction, \
    area_inertia, exact, mean_prediction = \
    analyze_single_file_experiment(H_values, data_dir, var_names, None, data_std,
                                   window_dur, window_step, area_ID,
                                   data_file_suffix='ieee39_PF_stoch_loads_compensators_split_vsg' + \
                                   f'_Pfrac={Pfrac:.1f}')

N_blocks = len(H_values)
block_dur = np.ceil(data_time[-1]) / N_blocks
plot_single_file_experiment(prediction_time, prediction, exact, mean_prediction,
                            N_blocks, block_dur, area_measure, measure_units)

EXPERIMENTS.append(make_experiment_dict(H_values, data_time, data, data_normalized,
                                        prediction_time, prediction, area_inertia, exact, mean_prediction,
                                       'momentum steps varying inertia of G2, G3 and VSG3'))

Same as before but with the generator `G03` subdivided in two parts, the second one modeled with either a type 2 or type 6 machine.

In [ ]:
db['experiments'] = EXPERIMENTS
db.close()